# Day 20 - o1-preview (GHC)

In [1]:
import collections

def read_input(filename):
    grid = []
    with open(filename) as f:
        for line in f:
            grid.append(line.rstrip("\n"))
    return grid

def bfs_distance(grid, start):
    rows, cols = len(grid), len(grid[0])
    dist = [[None]*cols for _ in range(rows)]
    q = collections.deque()
    (sr, sc) = start
    dist[sr][sc] = 0
    q.append((sr, sc))
    while q:
        r, c = q.popleft()
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0<=nr<rows and 0<=nc<cols and grid[nr][nc] != '#' and dist[nr][nc] is None:
                dist[nr][nc] = dist[r][c] + 1
                q.append((nr,nc))
    return dist

def solve_race(grid):
    rows, cols = len(grid), len(grid[0])
    start = end = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start = (r,c)
            elif grid[r][c] == 'E':
                end = (r,c)
    distS = bfs_distance(grid, start)
    distE = bfs_distance(grid, end)
    if distS[end[0]][end[1]] is None:
        return 0  # no path at all

    # Baseline time
    baseline = distS[end[0]][end[1]]

    # Precompute positions of '.' (track), including S/E
    track_positions = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] != '#':
                track_positions.append((r,c))

    # For each track position c1, gather possible c2 reachable in up to 2 ignoring walls
    # ignoring walls means we don't check grid[r][c] != '#' when we move
    # but we do require c2 to be on track in the end
    # We'll do a small BFS from each c1 allowing up to 2 steps ignoring walls.
    cheats_count = 0
    for (r1, c1) in track_positions:
        dS = distS[r1][c1]
        if dS is None:
            continue
        # BFS ignoring walls for up to 2 steps
        ignored_dist = [[-1]*cols for _ in range(rows)]
        ignored_dist[r1][c1] = 0
        queue = collections.deque([(r1, c1)])
        possible_ends = []
        while queue:
            rr, cc = queue.popleft()
            curd = ignored_dist[rr][cc]
            if curd<2:  # we can still move
                for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                    nr, nc = rr+dr, cc+dc
                    if 0<=nr<rows and 0<=nc<cols and ignored_dist[nr][nc]<0:
                        ignored_dist[nr][nc] = curd+1
                        queue.append((nr, nc))

        # Now see all positions with ignored_dist <=2 that are '.' so that cheat ends on track
        for r2 in range(rows):
            for c2 in range(cols):
                steps = ignored_dist[r2][c2]
                if steps>=0 and steps<=2 and grid[r2][c2] != '#':
                    dE = distE[r2][c2]
                    if dE is not None:
                        cost = dS + steps + dE
                        if baseline - cost >= 100:
                            cheats_count += 1

    return cheats_count

def main():
    grid = read_input("input.txt")
    result = solve_race(grid)
    print(result)

if __name__ == "__main__":
    main()

1402


## Part 2

In [2]:
import collections

def read_input(filename):
    grid = []
    with open(filename) as f:
        for line in f:
            grid.append(line.rstrip("\n"))
    return grid

def bfs_distance(grid, start):
    rows, cols = len(grid), len(grid[0])
    dist = [[None]*cols for _ in range(rows)]
    q = collections.deque()
    (sr, sc) = start
    dist[sr][sc] = 0
    q.append((sr, sc))
    while q:
        r, c = q.popleft()
        for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
            nr, nc = r+dr, c+dc
            if 0<=nr<rows and 0<=nc<cols and grid[nr][nc] != '#' and dist[nr][nc] is None:
                dist[nr][nc] = dist[r][c] + 1
                q.append((nr,nc))
    return dist

def solve_race(grid):
    rows, cols = len(grid), len(grid[0])
    start = end = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start = (r,c)
            elif grid[r][c] == 'E':
                end = (r,c)

    distS = bfs_distance(grid, start)
    distE = bfs_distance(grid, end)
    if distS[end[0]][end[1]] is None:
        return 0

    baseline = distS[end[0]][end[1]]
    track_positions = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] != '#':
                track_positions.append((r,c))

    # We only count each (start_pos, end_pos) once
    seen = set()
    for (r1, c1) in track_positions:
        dS = distS[r1][c1]
        if dS is None:
            continue

        # BFS ignoring walls for up to 20 steps
        ign_dist = [[-1]*cols for _ in range(rows)]
        ign_dist[r1][c1] = 0
        queue = collections.deque([(r1, c1)])
        while queue:
            rr, cc = queue.popleft()
            curd = ign_dist[rr][cc]
            if curd < 20:
                for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                    nr, nc = rr+dr, cc+dc
                    if 0<=nr<rows and 0<=nc<cols and ign_dist[nr][nc] < 0:
                        ign_dist[nr][nc] = curd + 1
                        queue.append((nr, nc))

        # For each track cell c2 that can be reached in up to 20 steps ignoring walls
        for r2 in range(rows):
            for c2 in range(cols):
                steps = ign_dist[r2][c2]
                if steps >= 0 and steps <= 20 and grid[r2][c2] != '#':
                    dE = distE[r2][c2]
                    if dE is not None:
                        cost = dS + steps + dE
                        saving = baseline - cost
                        if saving >= 100:
                            seen.add(((r1,c1),(r2,c2)))

    return len(seen)

def main():
    grid = read_input("input.txt")
    result = solve_race(grid)
    print(result)

if __name__ == "__main__":
    main()

1020244
